(model-monitoring-tutorial)= 
# Model Monitoring and Drift Detection

In this tutorial, we will be leveraging the model monitoring capabilities of MLRun to deploy a model to a live endpoint and calculate data drift.

Make sure you have reviewed the basics in MLRun [**Quick Start Tutorial**](../01-mlrun-basics.html).

Tutorial steps:
- [**Create an MLRun project**](#setup-project)
- [**Log a Model with a given framework and training set**](#log-model-with-training-data)
- [**Import an MLRun function from the marketplace**](#import-serving-function)
- [**Deploy Serving Function with Drift Detection**](#deploy-serving-function-with-drift-detection)
- [**Simulate Production Traffic**](#simulate-production-traffic)
- [**View Drift Calculations and Status**](#view-drift-calculations-and-status)
- [**View Detailed Drift Dashboards**](#view-detailed-drift-dashboards)

## MLRun Installation and Configuration

Before running this notebook make sure `mlrun` is installed and that you have configured the access to the MLRun service. 

In [ ]:
# install MLRun if not installed, run this only once (restart the notebook after the install !!!)
%pip install mlrun

## Setup Project

First, we will import the dependencies and create an [MLRun project](https://docs.mlrun.org/en/latest/projects/project.html). This will contain all of our models, functions, datasets, etc:

In [1]:
import mlrun
import os
import pandas as pd

> 2022-09-20 15:35:11,602 [info] Server and client versions are not the same: {'parsed_server_version': VersionInfo(major=1, minor=0, patch=4, prerelease=None, build=None), 'parsed_client_version': VersionInfo(major=1, minor=1, patch=0, prerelease=None, build=None)}


In [2]:
project = mlrun.get_or_create_project(name="drift-detection", context="./")

> 2022-09-20 15:35:31,185 [info] loaded project drift-detection from MLRun DB


> Note: This tutorial will not focus on training a model, but rather will start from the point of already having a trained model with a corresponding training dataset.

We will be logging the following model file and dataset to deploy and calculate data drift. The model is a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) from sklearn and the dataset is in `csv` format.

In [3]:
model_path = mlrun.get_sample_path('models/model-monitoring/model.pkl')
training_set_path = mlrun.get_sample_path('data/model-monitoring/train.csv')

## Log Model with Training Data

Next, we will log the model using MLRun experiment tracking. This is usually done in a training pipeline, but you can also bring in your pre-trained models from other sources. See [Working with data and model artifacts](https://docs.mlrun.org/en/latest/training/working-with-data-and-model-artifacts.html) and [Automated experiment tracking](https://docs.mlrun.org/en/latest/concepts/auto-logging-mlops.html) for more information.

In [4]:
model_name = "RandomForestClassifier"

In [5]:
model_artifact = project.log_model(
    key=model_name,
    model_file=model_path,
    framework="sklearn",
    training_set=pd.read_csv(training_set_path)
)

In [6]:
model_artifact.uri

'store://models/drift-detection/RandomForestClassifier#0:latest'

## Import Serving Function

Then, we will import the [model server](https://github.com/mlrun/functions/tree/master/v2_model_server) function from the [MLRun Function Marketplace](https://www.mlrun.org/marketplace/). Additionally, we will mount the filesytem, add our model that was logged via experiment tracking, and enable drift detection.

The core line here is `serving_fn.set_tracking()` which will create the required infrastructure behind the scenes to perform drift detection. See the [Model monitoring overview](https://docs.mlrun.org/en/latest/model_monitoring/model-monitoring-deployment.html) for more info on what is deployed.

In [7]:
# Import the serving function from the function hub and mount filesystem
serving_fn = mlrun.import_function('hub://v2_model_server', new_name="serving").apply(mlrun.auto_mount())

# Add the model to the serving function's routing spec
serving_fn.add_model(model_name, model_path=model_artifact.uri)

# Enable model monitoring
serving_fn.set_tracking()

## Deploy Serving Function with Drift Detection

Finally, we can deploy our serving function with drift detection enabled with a single line of code:

In [8]:
serving_fn.deploy()

> 2022-09-20 13:50:37,662 [info] Starting remote function deploy
2022-09-20 13:50:39  (info) Deploying function
2022-09-20 13:50:39  (info) Building
2022-09-20 13:50:40  (info) Staging files and preparing base images
2022-09-20 13:50:40  (info) Building processor image
2022-09-20 13:51:55  (info) Build complete
2022-09-20 13:52:09  (info) Function deploy complete
> 2022-09-20 13:52:10,139 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-drift-detection-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['drift-detection-serving-drift-detection.default-tenant.app.us-sales-350.iguazio-cd1.com/']}


'http://drift-detection-serving-drift-detection.default-tenant.app.us-sales-350.iguazio-cd1.com/'

## View Deployed Resources

At this point, you should see the newly deployed model server as well as a `model-monitoring-stream` and a scheduled job. The `model-monitoring-stream` will collect, process, and save the incoming requests to the model server. The scheduled job will do the actual calculation (by default every hour).

![drift_table_plot](../../_static/images/tutorial/model_monitoring/project_dashboard.png)

## Simulate Production Traffic

Next, we will use following code to simulate incoming production data using elements from the training set. Because the data is coming from the same training set we logged, we are not expecting any data drift.

> *Note: By default, the drift calculation will start via the scheduled hourly batch job after receiving 10,000 incoming requests*

In [ ]:
import json
from time import sleep
from random import choice, uniform
import logging
from tqdm import tqdm

# Suppress print messages
logging.getLogger(name="mlrun").setLevel(logging.WARNING)

# Get training set as list
iris_data = pd.read_csv(training_set_path).to_dict(orient="split")["data"]

# Simulate traffic using random elements from training set
for i in tqdm(range(12_000)):
    data_point = choice(iris_data)
    serving_fn.invoke(f'v2/models/{model_name}/infer', json.dumps({'inputs': [data_point]}))

## View Drift Calculations and Status

Once data drift has been calculated, you can view it in the MLRun UI. This includes a high level overview of the model status:

![model_endpoint_1](../../_static/images/tutorial/model_monitoring/model_endpoint_1.png)

A more detailed view on model information and overall drift metrics:

![model_endpoint_2](../../_static/images/tutorial/model_monitoring/model_endpoint_2.png)

As well as| a view for feature-level distributions and drift metrics:

![model_endpoint_3](../../_static/images/tutorial/model_monitoring/model_endpoint_3.png)

## View Detailed Drift Dashboards

Finally, there are also more detailed Grafana dashboards that show additional information on each model in the project:

> For more information on accessing these dashboards, see [Model monitoring using Grafana dashboards](https://docs.mlrun.org/en/latest/model_monitoring/model-monitoring-deployment.html#model-monitoring-using-grafana-dashboards).

![grafana_dashboard_1](../../_static/images/tutorial/model_monitoring/grafana_dashboard_1.png)

Graphs of individual features over time:

![grafana_dashboard_2](../../_static/images/tutorial/model_monitoring/grafana_dashboard_2.png)

As well as drift and operational metrics over time:

![grafana_dashboard_3](../../_static/images/tutorial/model_monitoring/grafana_dashboard_3.png)